## **Dataset Description**
## Dataset Name : 1000000 Sales Records
## Columns:
- Region : (ex: Africa , Middle East , Asia , Central America , etc).
- Country.
- Item Type : Type of product such as Fruits , Clothes , Meat ,etc.
- Sales Channel : Whether the transaction is done online or offline.
- Order Priority : The priority of the order (L:Low , M:Medium , H:High , C:Critical).
- Order Date : Date of ordering the product.
- Order ID : Unique identifier for each order.
- Ship Date : Date of product arrival.
- Units Sold : Number of items sold in this transaction.
- Unit Price : Price of each single product.
- Unit Cost : Cost of each single product.
- Total Revenue.
- Total Cost.
- Total Profit : Net Profit.

In [ ]:
!pip install pyspark

In [ ]:
import pandas as pd
import numpy as np
import pyspark
from pyspark.sql import SparkSession, SQLContext
from pyspark.mllib.stat import Statistics
#from pyspark.sql.functions import col, count ,isnan , when ,datediff ,date_format , to_date , to_timestamp ,lit
from pyspark.mllib.feature import StandardScaler
from pyspark.ml import Pipeline
from pyspark.ml.feature import OneHotEncoder , StringIndexer , VectorAssembler
from pyspark.ml.classification import LogisticRegression , RandomForestClassifier ,DecisionTreeClassifier , NaiveBayes
from pyspark.ml.evaluation import MulticlassClassificationEvaluator
from pyspark.sql.functions import *
from pyspark.sql import Window

## **Load Data**

In [ ]:
spark=SparkSession.builder.config("spark.driver.memory","15g").appName("Classification").getOrCreate()

In [ ]:
df=spark.read.csv("/content/drive/MyDrive/1000000 Sales Records.csv",header=True)

In [ ]:
df.show()

+--------------------+----------------+---------------+-------------+--------------+----------+---------+----------+----------+----------+---------+-------------+----------+------------+
|              Region|         Country|      Item Type|Sales Channel|Order Priority|Order Date| Order ID| Ship Date|Units Sold|Unit Price|Unit Cost|Total Revenue|Total Cost|Total Profit|
+--------------------+----------------+---------------+-------------+--------------+----------+---------+----------+----------+----------+---------+-------------+----------+------------+
|  Sub-Saharan Africa|    South Africa|         Fruits|      Offline|             M| 7/27/2012|443368995| 7/28/2012|      1593|      9.33|     6.92|     14862.69|  11023.56|     3839.13|
|Middle East and N...|         Morocco|        Clothes|       Online|             M| 9/14/2013|667593514|10/19/2013|      4611|    109.28|    35.84|    503890.08| 165258.24|   338631.84|
|Australia and Oce...|Papua New Guinea|           Meat|      Offl

## **Displaying the types of Dataframe columns**

In [ ]:
df.printSchema()

root
 |-- Region: string (nullable = true)
 |-- Country: string (nullable = true)
 |-- Item Type: string (nullable = true)
 |-- Sales Channel: string (nullable = true)
 |-- Order Priority: string (nullable = true)
 |-- Order Date: string (nullable = true)
 |-- Order ID: string (nullable = true)
 |-- Ship Date: string (nullable = true)
 |-- Units Sold: string (nullable = true)
 |-- Unit Price: string (nullable = true)
 |-- Unit Cost: string (nullable = true)
 |-- Total Revenue: string (nullable = true)
 |-- Total Cost: string (nullable = true)
 |-- Total Profit: string (nullable = true)



## **Converting dataframe numerical columns to float data type instead of string**

In [ ]:
numerical_columns=['Units Sold','Unit Price','Unit Cost','Total Revenue','Total Cost','Total Profit']

for column in numerical_columns:
  df=df.withColumn(column,df[column].cast("float").alias(column))


In [ ]:
df.printSchema()

root
 |-- Region: string (nullable = true)
 |-- Country: string (nullable = true)
 |-- Item Type: string (nullable = true)
 |-- Sales Channel: string (nullable = true)
 |-- Order Priority: string (nullable = true)
 |-- Order Date: string (nullable = true)
 |-- Order ID: string (nullable = true)
 |-- Ship Date: string (nullable = true)
 |-- Units Sold: float (nullable = true)
 |-- Unit Price: float (nullable = true)
 |-- Unit Cost: float (nullable = true)
 |-- Total Revenue: float (nullable = true)
 |-- Total Cost: float (nullable = true)
 |-- Total Profit: float (nullable = true)



In [ ]:
df.show()

+--------------------+----------------+---------------+-------------+--------------+----------+---------+----------+----------+----------+---------+-------------+----------+------------+
|              Region|         Country|      Item Type|Sales Channel|Order Priority|Order Date| Order ID| Ship Date|Units Sold|Unit Price|Unit Cost|Total Revenue|Total Cost|Total Profit|
+--------------------+----------------+---------------+-------------+--------------+----------+---------+----------+----------+----------+---------+-------------+----------+------------+
|  Sub-Saharan Africa|    South Africa|         Fruits|      Offline|             M| 7/27/2012|443368995| 7/28/2012|    1593.0|      9.33|     6.92|     14862.69|  11023.56|     3839.13|
|Middle East and N...|         Morocco|        Clothes|       Online|             M| 9/14/2013|667593514|10/19/2013|    4611.0|    109.28|    35.84|     503890.1| 165258.23|   338631.84|
|Australia and Oce...|Papua New Guinea|           Meat|      Offl

## **Checking for Nulls in each column of the dataframe**

In [ ]:
df.select([count(when(col(c).isNull(),c)).alias(c) for c in df.columns]).show()

+------+-------+---------+-------------+--------------+----------+--------+---------+----------+----------+---------+-------------+----------+------------+
|Region|Country|Item Type|Sales Channel|Order Priority|Order Date|Order ID|Ship Date|Units Sold|Unit Price|Unit Cost|Total Revenue|Total Cost|Total Profit|
+------+-------+---------+-------------+--------------+----------+--------+---------+----------+----------+---------+-------------+----------+------------+
|     0|      0|        0|            0|             0|         0|       0|        0|         0|         0|        0|            0|         0|           0|
+------+-------+---------+-------------+--------------+----------+--------+---------+----------+----------+---------+-------------+----------+------------+



## **Checking if there are duplicate rows in the dataframe**

In [ ]:
print(df.count()-df.distinct().count())

50009


## **Removing duplicate records in the dataframe**

In [ ]:
df=df.dropDuplicates()

In [ ]:
df.show()

+--------------------+----------------+---------------+-------------+--------------+----------+---------+----------+----------+----------+---------+-------------+----------+------------+
|              Region|         Country|      Item Type|Sales Channel|Order Priority|Order Date| Order ID| Ship Date|Units Sold|Unit Price|Unit Cost|Total Revenue|Total Cost|Total Profit|
+--------------------+----------------+---------------+-------------+--------------+----------+---------+----------+----------+----------+---------+-------------+----------+------------+
|  Sub-Saharan Africa|          Malawi|      Household|      Offline|             H| 4/24/2012|358857953| 5/14/2012|    2898.0|    668.27|   502.54|    1936646.5| 1456360.9|   480285.53|
|                Asia|           Nepal|  Personal Care|       Online|             H| 5/20/2011|543531262|  6/3/2011|    1834.0|     81.73|    56.67|    149892.81| 103932.78|    45960.04|
|Middle East and N...|         Algeria|         Fruits|      Offl

## **Converting Order Date and Ship Date to MM/dd/yyyy Format**

In [ ]:
from pyspark.sql.functions import *
def convert_date_to_correct_format(date_col,col_name):
  date_col_updated=[]
  for elem in date_col.collect():
    date_components=elem[col_name].split('/')
    reformatted_date=""
    if(len(date_components[0])==1):
      reformatted_date+="0"
    reformatted_date+=date_components[0]
    reformatted_date+="/"
    if(len(date_components[1])==1):
      reformatted_date+="0"
    reformatted_date+=date_components[1]
    reformatted_date+="/"
    reformatted_date+=date_components[2]
    date_col_updated.append(reformatted_date)
  return date_col_updated

order_date_column =df.select(col("Order Date"))
ship_date_column=df.select(col("Ship Date"))

reformatted_order_date=convert_date_to_correct_format(order_date_column,"Order Date")
reformatted_ship_date=convert_date_to_correct_format(ship_date_column,"Ship Date")



In [ ]:
c1 = spark.createDataFrame([(l,) for l in reformatted_order_date], ['Reformatted Order Date'])
c2 = spark.createDataFrame([(l,) for l in reformatted_ship_date], ['Reformatted Ship Date'])
c1= c1.withColumn("row_idx", row_number().over(Window.orderBy(monotonically_increasing_id())))
c2 =c2.withColumn("row_idx", row_number().over(Window.orderBy(monotonically_increasing_id())))
dates_df = c1.join(c2, c1.row_idx == c2.row_idx).drop(c1.row_idx)
df=df.withColumn("row_idx", row_number().over(Window.orderBy(monotonically_increasing_id())))
df=df.join(dates_df,df.row_idx == dates_df.row_idx).drop('row_idx')
df.show()

+--------------------+----------------+---------------+-------------+--------------+----------+---------+----------+----------+----------+---------+-------------+----------+------------+----------------------+---------------------+
|              Region|         Country|      Item Type|Sales Channel|Order Priority|Order Date| Order ID| Ship Date|Units Sold|Unit Price|Unit Cost|Total Revenue|Total Cost|Total Profit|Reformatted Order Date|Reformatted Ship Date|
+--------------------+----------------+---------------+-------------+--------------+----------+---------+----------+----------+----------+---------+-------------+----------+------------+----------------------+---------------------+
|  Sub-Saharan Africa|          Malawi|      Household|      Offline|             H| 4/24/2012|358857953| 5/14/2012|    2898.0|    668.27|   502.54|    1936646.5| 1456360.9|   480285.53|            04/24/2012|           05/14/2012|
|                Asia|           Nepal|  Personal Care|       Online|   

## **Adding a new column to the dataframe called shipping days which is the difference between the Shipping date and Order date**

In [ ]:
df=df.select("*",
              datediff(to_date(col("Reformatted Ship Date"),"MM/dd/yyyy"),
                       to_date(col("Reformatted Order Date"),"MM/dd/yyyy")).alias("Shipping days"))
df.show()

+--------------------+----------------+---------------+-------------+--------------+----------+---------+----------+----------+----------+---------+-------------+----------+------------+----------------------+---------------------+-------------+
|              Region|         Country|      Item Type|Sales Channel|Order Priority|Order Date| Order ID| Ship Date|Units Sold|Unit Price|Unit Cost|Total Revenue|Total Cost|Total Profit|Reformatted Order Date|Reformatted Ship Date|Shipping days|
+--------------------+----------------+---------------+-------------+--------------+----------+---------+----------+----------+----------+---------+-------------+----------+------------+----------------------+---------------------+-------------+
|  Sub-Saharan Africa|          Malawi|      Household|      Offline|             H| 4/24/2012|358857953| 5/14/2012|    2898.0|    668.27|   502.54|    1936646.5| 1456360.9|   480285.53|            04/24/2012|           05/14/2012|           20|
|               

## **Dropping some columns after feature engineering and data cleaning and keeping only the columns that will be used during modeling**

In [ ]:
columns_to_drop=['Order ID','Total Revenue','Total Cost','Order Date','Ship Date','Reformatted Order Date','Reformatted Ship Date']
df=df.drop(*(columns_to_drop))

## **Label encoding for the output column (Order Priority)**

In [ ]:
indexer=StringIndexer(inputCol="Item Type",outputCol="Item Type Label").fit(df)
df=indexer.transform(df)
df.show()

+--------------------+----------------+---------------+-------------+--------------+----------+----------+---------+------------+-------------+---------------+
|              Region|         Country|      Item Type|Sales Channel|Order Priority|Units Sold|Unit Price|Unit Cost|Total Profit|Shipping days|Item Type Label|
+--------------------+----------------+---------------+-------------+--------------+----------+----------+---------+------------+-------------+---------------+
|  Sub-Saharan Africa|          Malawi|      Household|      Offline|             H|    2898.0|    668.27|   502.54|   480285.53|           20|            7.0|
|                Asia|           Nepal|  Personal Care|       Online|             H|    1834.0|     81.73|    56.67|    45960.04|           14|            1.0|
|Middle East and N...|         Algeria|         Fruits|      Offline|             C|     154.0|      9.33|     6.92|      371.14|           43|            0.0|
|  Sub-Saharan Africa|        Tanzania| 

## **Feature engineering the categorical columns using one hot encoding technique**

In [ ]:
## Showing the number of unique values for each categorical column 
text_columns=["Region","Country","Item Type","Sales Channel"]
for col in text_columns:
  print(col+" has "+str(df.select(col).distinct().count())+" Distinct Values")

Region has 7 Distinct Values
Country has 185 Distinct Values
Item Type has 12 Distinct Values
Sales Channel has 2 Distinct Values


In [ ]:
####
text_columns=["Region","Country","Order Priority","Sales Channel"]
indexers=[StringIndexer(inputCol=c,outputCol="{}_indexed".format(c)) for c in text_columns]
encoders = [OneHotEncoder(inputCol=indexer.getOutputCol(),
                        outputCol="{}_encoded".format(indexer.getOutputCol()))
                        for indexer in indexers]

all_columns=[]
filtered_numerical_columns=['Units Sold','Unit Price','Unit Cost','Shipping days']
all_columns.extend(filtered_numerical_columns)
encoder_columns=[encoder.getOutputCol() for encoder in encoders]
all_columns.extend(encoder_columns)

print(all_columns)
assembler = VectorAssembler(
    inputCols=all_columns,
    outputCol="features"
)
pipeline = Pipeline(stages=indexers + encoders + [assembler])
final_features=pipeline.fit(df).transform(df)
final_features.show()

['Units Sold', 'Unit Price', 'Unit Cost', 'Shipping days', 'Region_indexed_encoded', 'Country_indexed_encoded', 'Order Priority_indexed_encoded', 'Sales Channel_indexed_encoded']
+--------------------+----------------+---------------+-------------+--------------+----------+----------+---------+------------+-------------+---------------+--------------+---------------+----------------------+---------------------+----------------------+-----------------------+------------------------------+-----------------------------+--------------------+
|              Region|         Country|      Item Type|Sales Channel|Order Priority|Units Sold|Unit Price|Unit Cost|Total Profit|Shipping days|Item Type Label|Region_indexed|Country_indexed|Order Priority_indexed|Sales Channel_indexed|Region_indexed_encoded|Country_indexed_encoded|Order Priority_indexed_encoded|Sales Channel_indexed_encoded|            features|
+--------------------+----------------+---------------+-------------+--------------+-------

In [ ]:
final_features.select("features","Item Type Label").show()

+--------------------+---------------+
|            features|Item Type Label|
+--------------------+---------------+
|(198,[0,1,2,3,4,7...|            7.0|
|(198,[0,1,2,3,6,9...|            1.0|
|(198,[0,1,2,3,7,1...|            0.0|
|(198,[0,1,2,3,4,5...|            3.0|
|(198,[0,1,2,3,4,1...|            3.0|
|(198,[0,1,2,3,5,1...|            4.0|
|(198,[0,1,2,3,4,7...|            5.0|
|(198,[0,1,2,3,4,2...|            2.0|
|(198,[0,1,2,3,9,1...|           10.0|
|(198,[0,1,2,3,7,6...|            8.0|
|(198,[0,1,2,3,5,1...|            4.0|
|(198,[0,1,2,3,8,1...|            3.0|
|(198,[0,1,2,3,9,1...|            1.0|
|(198,[0,1,2,3,6,3...|           11.0|
|(198,[0,1,2,3,8,8...|            3.0|
|(198,[0,1,2,3,5,6...|            4.0|
|(198,[0,1,2,3,8,1...|            8.0|
|(198,[0,1,2,3,4,2...|            2.0|
|(198,[0,1,2,3,99,...|            4.0|
|(198,[0,1,2,3,5,2...|            2.0|
+--------------------+---------------+
only showing top 20 rows



## **Splitting the data into 80% training and 20% testing**

In [ ]:
train_df , test_df= final_features.randomSplit([0.8,0.2])

## **Logistic Regression**

In [ ]:
model=LogisticRegression(featuresCol='features',labelCol='Item Type Label')
LR_model=model.fit(train_df)

In [ ]:
preds_train=LR_model.transform(train_df)
preds_test=LR_model.transform(test_df)
evaluator=MulticlassClassificationEvaluator(labelCol="Item Type Label",predictionCol="prediction",metricName="accuracy")
print("Accuracy on training data is: "+str(evaluator.evaluate(preds_train)))
print("Accuracy on testing data is: "+str(evaluator.evaluate(preds_test)))


Accuracy on training data is: 1.0
Accuracy on testing data is: 1.0


## **Decision Tree Classifier**

In [ ]:
model=DecisionTreeClassifier(featuresCol='features',labelCol='Item Type Label',maxDepth=10)
DT_model=model.fit(train_df)

In [ ]:
preds_train=DT_model.transform(train_df)
preds_test=DT_model.transform(test_df)
evaluator=MulticlassClassificationEvaluator(labelCol="Item Type Label",predictionCol="prediction",metricName="accuracy")
print("Accuracy on training data is: "+str(evaluator.evaluate(preds_train)))
print("Accuracy on testing data is: "+str(evaluator.evaluate(preds_test)))

Accuracy on training data is: 1.0
Accuracy on testing data is: 1.0
